In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


# Presentation to be like "Time Varying Liquidity and Momentum Profits (2015) paper

Rank by deciles, HML (8,9,10) - (1,2,3).  Save monthly returns (equal (risk?) weighted) and if you want to annualized then resample.


Then correlation matrix 
* Overall market vol?
* Market illiquidity (median Amihud through time series)


# Amihud (2002) Annual portfolio sorts

In [6]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

tsmom_model.py:275: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
tsmom_model.py:275: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=3,window=100000000,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


# Annual portfolio sorts

In [7]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,3)
sector_portfolio_sorts['All']=quantile_portfolios_annual(data,cleansed,3)

tsmom_model.py:240: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
tsmom_model.py:241: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


In [8]:
for sec in sector_portfolio_sorts.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())


Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.69 &   0.76 &   0.95 &    0.27 \\
Standard Deviation           &  17.26 &  14.73 &  17.69 &   17.06 \\
Information Ratio            &   0.48 &   0.62 &   0.65 &    0.19 \\
Skewness                     &  -0.01 &  -0.11 &   0.91 &    0.32 \\
Excess Kurtosis              &   1.23 &   1.19 &   5.63 &    2.43 \\
CAPM Alpha Annualized (in \%) &   4.30 &   5.89 &   7.62 &    3.32 \\
CAPM Alpha Tstat             &   2.15 &   2.95 &   3.34 &    0.91 \\
CAPM Beta (in \%)             &   1.08 &   0.90 &   1.05 &   -0.03 \\
CAPM Beta Tstat              &  19.10 &  21.64 &  13.46 &   -0.27 \\
\$R\textasciicircum2\$                        &   0.72 &   0.68 &   0.65 &    0.00 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.36 &   0.55 &   0.69 &    0.34 \\
Standard Deviation   

# FHT Annual portfolio sorts

In [9]:
FHT=calculate_FHT(cleansed)
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(FHT)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

tsmom_model.py:288: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
tsmom_model.py:289: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
tsmom_model.py:290: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
tsmom_model.py:290: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')


In [10]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,3)
sector_portfolio_sorts_FHT['All']=quantile_portfolios_annual(data,cleansed,3)

In [11]:
for sec in sector_portfolio_sorts_FHT.keys():
    print(sec)
    print(portfolio_sort_table(sector_portfolio_sorts_FHT[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.52 &   0.78 &   0.51 &   -0.01 \\
Standard Deviation           &  14.18 &  16.87 &  16.10 &   15.24 \\
Information Ratio            &   0.44 &   0.55 &   0.38 &   -0.00 \\
Skewness                     &  -0.14 &   1.49 &  -0.06 &    0.17 \\
Excess Kurtosis              &   1.63 &  11.56 &   0.58 &    0.07 \\
CAPM Alpha Annualized (in \%) &   5.26 &   7.66 &   4.71 &   -0.54 \\
CAPM Alpha Tstat             &   2.74 &   2.99 &   2.36 &   -0.16 \\
CAPM Beta (in \%)             &   0.96 &   1.01 &   1.05 &    0.09 \\
CAPM Beta Tstat              &  20.23 &  10.36 &  22.00 &    1.17 \\
\$R\textasciicircum2\$                        &   0.70 &   0.61 &   0.72 &    0.00 \\
\bottomrule
\end{tabular}

All


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  This is separate from the ipykernel package so we can avoid doing imports until


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.33 &   0.51 &   0.56 &    0.23 \\
Standard Deviation           &   7.44 &   8.37 &   9.91 &    8.11 \\
Information Ratio            &   0.53 &   0.74 &   0.68 &    0.35 \\
Skewness                     &  -0.94 &  -0.69 &  -0.59 &    0.01 \\
Excess Kurtosis              &   5.70 &   3.34 &   3.24 &    0.23 \\
CAPM Alpha Annualized (in \%) &   3.03 &   4.36 &   3.79 &    0.76 \\
CAPM Alpha Tstat             &   3.31 &   4.77 &   3.55 &    0.44 \\
CAPM Beta (in \%)             &   0.87 &   0.97 &   1.09 &    0.22 \\
CAPM Beta Tstat              &  28.44 &  23.44 &  22.95 &    3.19 \\
\$R\textasciicircum2\$                        &   0.79 &   0.83 &   0.81 &    0.06 \\
\bottomrule
\end{tabular}

Currencies
\begin{tabular}{lrrrr}
\toprule
{} &     0 &     1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        & -0.10 & -0.04 &   0.01 &    0.11 \\
Standard Deviation

# Monthly portoflio sorts

In [13]:
amihud_monthly = read_monthly(amihud=True,sorts=3)

In [14]:
for sec in amihud_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(amihud_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(amihud_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())

Agriculturals
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.80 &   0.69 &   0.68 &   -0.12 \\
Standard Deviation           &  16.56 &  14.49 &  18.61 &   16.00 \\
Information Ratio            &   0.58 &   0.57 &   0.44 &   -0.09 \\
Skewness                     &  -0.05 &   0.04 &   0.71 &    0.38 \\
Excess Kurtosis              &   1.76 &   0.44 &   5.57 &    1.78 \\
CAPM Alpha Annualized (in \%) &   6.41 &   5.49 &   4.51 &   -1.90 \\
CAPM Alpha Tstat             &   3.40 &   2.63 &   2.20 &   -0.60 \\
CAPM Beta (in \%)             &   1.01 &   0.87 &   1.15 &    0.14 \\
CAPM Beta Tstat              &  19.08 &  17.86 &  14.81 &    1.23 \\
\$R\textasciicircum2\$                        &   0.71 &   0.69 &   0.73 &    0.01 \\
\bottomrule
\end{tabular}

All


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.42 &   0.37 &   0.58 &    0.16 \\
Standard Deviation           &   9.81 &   8.73 &  10.34 &    8.32 \\
Information Ratio            &   0.51 &   0.52 &   0.67 &    0.23 \\
Skewness                     &  -0.75 &  -1.06 &  -0.26 &   -0.01 \\
Excess Kurtosis              &   4.40 &   3.98 &   2.61 &    0.83 \\
CAPM Alpha Annualized (in \%) &   3.14 &   2.78 &   4.98 &    1.84 \\
CAPM Alpha Tstat             &   2.94 &   3.35 &   4.54 &    0.99 \\
CAPM Beta (in \%)             &   1.00 &   0.91 &   1.04 &    0.04 \\
CAPM Beta Tstat              &  24.45 &  28.45 &  18.96 &    0.55 \\
\$R\textasciicircum2\$                        &   0.79 &   0.82 &   0.77 &    0.00 \\
\bottomrule
\end{tabular}

Currencies
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.01 &   0.00 &  -0.01 &   -0.02 \\
Standard Devia

# Monthly FHT

In [16]:
FHT_monthly = read_monthly(amihud=False,sorts=3)

In [17]:
for sec in FHT_monthly.keys():
    print(sec)
    #print(portfolio_sort_table(FHT_monthly[sec],sector_rtns[sec]).to_latex())
    print(portfolio_sort_table(FHT_monthly[sec],np.log(cleansed.resample(rule='m',how='last')).diff().dropna(how='all'),d_map,sec).to_latex())



Agriculturals


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.96 &   0.47 &   0.24 &   -0.72 \\
Standard Deviation           &  13.71 &  14.20 &  17.44 &   16.35 \\
Information Ratio            &   0.84 &   0.39 &   0.17 &   -0.53 \\
Skewness                     &  -0.04 &  -0.10 &   0.56 &    0.57 \\
Excess Kurtosis              &   0.52 &   1.35 &   6.24 &    2.96 \\
CAPM Alpha Annualized (in \%) &  11.62 &   4.25 &   0.68 &  -10.95 \\
CAPM Alpha Tstat             &   5.78 &   2.12 &   0.29 &   -2.97 \\
CAPM Beta (in \%)             &   0.85 &   0.97 &   1.17 &    0.32 \\
CAPM Beta Tstat              &  16.74 &  21.47 &  15.92 &    3.02 \\
\$R\textasciicircum2\$                        &   0.65 &   0.71 &   0.71 &    0.06 \\
\bottomrule
\end{tabular}

All
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.54 &   0.40 &   0.52 &   -0.02 \\
Standard Deviation   

tsmom_model.py:323: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  res=sm.OLS(un_dec.dropna()[str(i)].loc[ind],capm_factor[['Intercept','Mkt-RF']].loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})


\begin{tabular}{lrrrr}
\toprule
{} &     0 &     1 &     2 &  Factor \\
\midrule
Monthly Return (in \%)        & -0.06 &  0.00 & -0.02 &    0.05 \\
Standard Deviation           &  9.01 &  7.60 &  7.47 &   10.71 \\
Information Ratio            & -0.08 &  0.00 & -0.03 &    0.05 \\
Skewness                     & -0.52 &  0.04 & -0.18 &    0.20 \\
Excess Kurtosis              &  1.65 &  1.53 &  4.16 &    2.84 \\
CAPM Alpha Annualized (in \%) &   NaN &   NaN &   NaN &     NaN \\
CAPM Alpha Tstat             &   NaN &   NaN &   NaN &     NaN \\
CAPM Beta (in \%)             &   NaN &   NaN &   NaN &     NaN \\
CAPM Beta Tstat              &   NaN &   NaN &   NaN &     NaN \\
\$R\textasciicircum2\$                        &   NaN &   NaN &   NaN &     NaN \\
\bottomrule
\end{tabular}

Energies
\begin{tabular}{lrrrr}
\toprule
{} &      0 &      1 &      2 &  Factor \\
\midrule
Monthly Return (in \%)        &   0.10 &   1.08 &   1.42 &    1.32 \\
Standard Deviation           &  29.91 &  29.48 & 